# Teraef
dawdwadawd

# Introduction

We are proud to share that we won QuEra’s challenge at the ETH Zurich Quantum Hackathon. The event spanned three intense and rewarding days. After the opening keynote, we were introduced to the five available challenges and had the opportunity to briefly visit ETH Zurich’s cutting-edge research facilities. Among the proposed topics, we were particularly drawn to QuEra’s challenge, which focused on optimizing quantum circuits for their neutral-atom quantum computing platform — a problem space that resonated strongly with our background in computer science.

On the second day, we met Pedro and John from the QuEra team, who presented the technical details of the task and provided valuable guidance throughout the competition. Once the challenge was fully explained, the 24-hour countdown began—and so did our focused effort.

As this was our first hackathon experience, we found it to be deeply educational, technically demanding, and ultimately very fulfilling. What set our team apart was a distinctive approach, different from that of most other participants, shaped by our perspective as computer scientists.

In this post, we share our experience and reflect on the process that led us to our final solution.
